<a href="https://colab.research.google.com/github/davanstrien/people-iiif/blob/main/01_wikidata_dl_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
!pip install httpx loguru fastcore sparqlwrapper tqdm

```sql
# ?theme ?themeLabel ?tgm ?item ?itemLabel ?url
select ?themeLabel ?tgm ?url {
  # Get themes that have LTGCM subject headings
  ?theme wdt:P5160 ?tgm .
  # Get the items that dipict the theme
  ?item wdt:P180 ?theme .
  # limit to Photographs
  ?item wdt:P31 wd:Q125191 .
  # Image url P6108
  ?item wdt:P18 ?url
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
   }
}
```

In [123]:
import pandas as pd
import httpx
import io
from pathlib import Path
from fastcore.all import *
from PIL import Image
import hashlib
from loguru import logger

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

In [124]:
from tqdm.notebook import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [125]:
path = Path('/content/drive/MyDrive/IIIF_AI_project')

In [161]:
endpoint_url = "https://query.wikidata.org/sparql"

endpoint_url = "https://query.wikidata.org/sparql"

query = """# ?theme ?themeLabel ?tgm ?item ?itemLabel ?url
select ?themeLabel ?tgm ?url ?item where {
  # Get themes that have LTGCM subject headings
  ?theme wdt:P5160 ?tgm .
  # Get the items that dipict the theme
  ?item wdt:P180 ?theme .
  # limit to Photographs
  ?item wdt:P31 wd:Q125191 .
  # Image url P6108
  ?item wdt:P18 ?url
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
   }
}  order by desc(?count) """


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)


In [186]:
df = pd.DataFrame.from_dict(results["results"]["bindings"])

In [187]:
df['item'] = df['item'].str['value']

In [188]:
df['tgm'] = df['tgm'].str['value']

In [189]:
df['themeLabel'] = df['themeLabel'].str['value']

In [190]:
df['url' ] = df['url'].str['value']

In [191]:
df['urlhash'] = pd.util.hash_pandas_object(df.url, index=False)

In [192]:
df

,tgm,item,themeLabel,url,urlhash
0,tgm002065,http://www.wikidata.org/entity/Q29384004,cigarette,http://commons.wikimedia.org/wiki/Special:File...,5476561517036506605
1,tgm002065,http://www.wikidata.org/entity/Q29578945,cigarette,http://commons.wikimedia.org/wiki/Special:File...,2547666618636415996
2,tgm002065,http://www.wikidata.org/entity/Q56713789,cigarette,http://commons.wikimedia.org/wiki/Special:File...,3108069741139737998
3,tgm002065,http://www.wikidata.org/entity/Q56718359,cigarette,http://commons.wikimedia.org/wiki/Special:File...,6536074540519139147
4,tgm002065,http://www.wikidata.org/entity/Q78123994,cigarette,http://commons.wikimedia.org/wiki/Special:File...,17560400556605834985
...,...,...,...,...,...
13819,tgm000686,http://www.wikidata.org/entity/Q75691200,award,http://commons.wikimedia.org/wiki/Special:File...,6543997110755914170
13820,tgm002207,http://www.wikidata.org/entity/Q78145666,coachman,http://commons.wikimedia.org/wiki/Special:File...,4983334255754292679
13821,tgm003584,http://www.wikidata.org/entity/Q96181582,work,http://commons.wikimedia.org/wiki/Special:File...,2158356187537040849
13822,tgm003584,http://www.wikidata.org/entity/Q96803013,work,http://commons.wikimedia.org/wiki/Special:File...,2470024329174724360


In [197]:
df[df.columns].groupby('item')['themeLabel'].apply(lambda x: ";".join(x)).reset_index()

,item,themeLabel
0,http://www.wikidata.org/entity/Q104213111,rain
1,http://www.wikidata.org/entity/Q104213115,architecture
2,http://www.wikidata.org/entity/Q104213117,tree
3,http://www.wikidata.org/entity/Q104213207,chair
4,http://www.wikidata.org/entity/Q104213209,library;architecture
...,...,...
12136,http://www.wikidata.org/entity/Q97989485,railway
12137,http://www.wikidata.org/entity/Q97989530,railway
12138,http://www.wikidata.org/entity/Q98823705,portrait
12139,http://www.wikidata.org/entity/Q98944658,bridge


In [198]:
df

,tgm,item,themeLabel,url,urlhash,label
0,tgm002065,http://www.wikidata.org/entity/Q29384004,cigarette,http://commons.wikimedia.org/wiki/Special:File...,5476561517036506605,cigarette
1,tgm002065,http://www.wikidata.org/entity/Q29578945,cigarette,http://commons.wikimedia.org/wiki/Special:File...,2547666618636415996,cigarette
2,tgm002065,http://www.wikidata.org/entity/Q56713789,cigarette,http://commons.wikimedia.org/wiki/Special:File...,3108069741139737998,cigarette;portrait
3,tgm002065,http://www.wikidata.org/entity/Q56718359,cigarette,http://commons.wikimedia.org/wiki/Special:File...,6536074540519139147,cigarette;portrait
4,tgm002065,http://www.wikidata.org/entity/Q78123994,cigarette,http://commons.wikimedia.org/wiki/Special:File...,17560400556605834985,cigarette
...,...,...,...,...,...,...
13819,tgm000686,http://www.wikidata.org/entity/Q75691200,award,http://commons.wikimedia.org/wiki/Special:File...,6543997110755914170,sport;award
13820,tgm002207,http://www.wikidata.org/entity/Q78145666,coachman,http://commons.wikimedia.org/wiki/Special:File...,4983334255754292679,coachman
13821,tgm003584,http://www.wikidata.org/entity/Q96181582,work,http://commons.wikimedia.org/wiki/Special:File...,2158356187537040849,work
13822,tgm003584,http://www.wikidata.org/entity/Q96803013,work,http://commons.wikimedia.org/wiki/Special:File...,2470024329174724360,work


In [200]:
df.to_csv(path/'wikidata_urls.csv', index=False)

In [201]:
df.to_json(path/'wikidata_urls.json')

In [204]:
Path(path/'wikidata-images-labels').mkdir(exist_ok=True)

In [205]:
save_path = Path(path/'wikidata-images-labels')

In [208]:
def download_image(row, path=save_path):
    url = row.url
    hash = row.urlhash
    ext = url.split('.')[-1]
    fpath = f'{path}/{hash}.{ext}'
    if not Path(fpath).is_file():
        try:
            r = httpx.get(url, timeout=30)
            im = Image.open(io.BytesIO(r.content))
            im.save(fpath)
            return fpath         
        except Exception as e:
            logger.exception(e)
    else:
        pass

In [209]:
df['dl_fpath'] = df.progress_apply(download_image, axis=1)

In [210]:
df = df[df.dl_fpath.notna()].copy(deep=True)

In [211]:
df

,tgm,item,themeLabel,url,urlhash,label,dl_fpath
0,tgm002065,http://www.wikidata.org/entity/Q29384004,cigarette,http://commons.wikimedia.org/wiki/Special:File...,5476561517036506605,cigarette,/content/drive/MyDrive/IIIF_AI_project/wikidat...
1,tgm002065,http://www.wikidata.org/entity/Q29578945,cigarette,http://commons.wikimedia.org/wiki/Special:File...,2547666618636415996,cigarette,/content/drive/MyDrive/IIIF_AI_project/wikidat...
2,tgm002065,http://www.wikidata.org/entity/Q56713789,cigarette,http://commons.wikimedia.org/wiki/Special:File...,3108069741139737998,cigarette;portrait,/content/drive/MyDrive/IIIF_AI_project/wikidat...
3,tgm002065,http://www.wikidata.org/entity/Q56718359,cigarette,http://commons.wikimedia.org/wiki/Special:File...,6536074540519139147,cigarette;portrait,/content/drive/MyDrive/IIIF_AI_project/wikidat...
4,tgm002065,http://www.wikidata.org/entity/Q78123994,cigarette,http://commons.wikimedia.org/wiki/Special:File...,17560400556605834985,cigarette,/content/drive/MyDrive/IIIF_AI_project/wikidat...
...,...,...,...,...,...,...,...
13817,tgm000686,http://www.wikidata.org/entity/Q75691041,award,http://commons.wikimedia.org/wiki/Special:File...,18274333717120672795,award,/content/drive/MyDrive/IIIF_AI_project/wikidat...
13820,tgm002207,http://www.wikidata.org/entity/Q78145666,coachman,http://commons.wikimedia.org/wiki/Special:File...,4983334255754292679,coachman,/content/drive/MyDrive/IIIF_AI_project/wikidat...
13821,tgm003584,http://www.wikidata.org/entity/Q96181582,work,http://commons.wikimedia.org/wiki/Special:File...,2158356187537040849,work,/content/drive/MyDrive/IIIF_AI_project/wikidat...
13822,tgm003584,http://www.wikidata.org/entity/Q96803013,work,http://commons.wikimedia.org/wiki/Special:File...,2470024329174724360,work,/content/drive/MyDrive/IIIF_AI_project/wikidat...


In [46]:
df.reset_index(inplace=True)

In [218]:
df.to_csv(path/'wikidata_w_image-labels.csv', index=False)

In [214]:
df.to_json(path/'wikidata_w_image-labels.json')